In [1]:
# %cd ../
# Install our library 
!pip install CanvasHacks

import environment
from assets.RequestTools import get_all_course_assignments, get_assignments_needing_grading
from assets.Configuration import InteractiveConfiguration
from assets.InputFields import make_course_ids_input, make_canvas_token_input, make_general_reset_button
import assets.GradingTools as GT
import assets.DownloadProcessingTools as PT
from assets.DataManagement import DataStore

/Users/adam/Dropbox/CanvasCorrections


# Setup the configuration

## Canvas API Token

You'll need to generate a token. This stands in for your username/password. 

Make sure you record the token in a secure place. It will not be saved in this notebook and you will have to re-enter it every time you use this tool.

https://canvas.instructure.com/doc/api/file.oauth.html#manual-token-generation


To manually generate a token for testing:

    Click the "profile" link in the top right menu bar, or navigate to /profile

    Under the "Approved Integrations" section, click the button to generate a new access token.
    
    Once the token is generated, you cannot view it again, and you'll have to generate a new token if you forget it. Remember that access tokens are password equivalent, so keep it secret.



In [ ]:
make_canvas_token_input()

## Course ids

You will also need the id number(s) of the courses you wish to grade. You can find them by logging into canvas and going to the course page. The number you are looking for will be in the url, immediately after '/courses/'

    For example, if the url is: https://canvas.csun.edu/courses/12345 
    
    The course number is: 12345
    
You will want to also write this number down somewhere secure since the values will be purged when you close the notebook.

Add the course ids one-at-a-time using the box and buttons below

In [9]:
make_course_ids_input()

# Figure out what needs grading

In [ ]:
print("These assignments need grading: ")
for sec in environment.CONFIG.course_ids:
    to_grade = get_assignments_needing_grading(sec)
    print(sec, to_grade)

# Download and process student submissions

Todo

    - Do we want to have more control over which assignments are graded? This just does all ungraded...

In [3]:
results = []


for course_id in environment.CONFIG.course_ids:
    print('course', course_id)
    to_grade = get_assignments_needing_grading(course_id)
    
    for name, assignment_id in to_grade:
        store = DataStore(assignment_id=assignment_id, assignment_name=name, course_id=course_id)

        print("Processing {}".format(name))
        # make folder
        # download student submissions 
        response = PT.get_submissions(course_id, assignment_id)
        print("{} responses received".format(len(response)))
        store.submissions = PT.process_response_without_saving_files(response)
        # give credit for non-empty submissions
        c = GT.determine_credit(store.submissions)
        # stow the results in our data object
        store.credit = c['credit']
        store.no_credit = c['nocredit']
        store.print_counts()
        # Add the now full-of-data object to our results list
        results.append(store)


course 67531
Processing Journal (week 1)
https://canvas.csun.edu/api/v1/courses/67531/assignments/360442/submissions?per_page=42
https://canvas.csun.edu/api/v1/courses/67531/assignments/360442/submissions?page=2&per_page=42
https://canvas.csun.edu/api/v1/courses/67531/assignments/360442/submissions?page=3&per_page=42
126 responses received
Downloading:  https://canvas.csun.edu/files/5617587/download?download_frd=1&verifier=ZAzG9KwDpyqhoMHtX9gttiUx5rQ54Dao6X6RTqEX
Downloading:  https://canvas.csun.edu/files/5588856/download?download_frd=1&verifier=8QnasIbl4LT5BC7DLG2AtSQA9kBghrgtrdlQgnnU
Downloading:  https://canvas.csun.edu/files/5619568/download?download_frd=1&verifier=8X8gXTPdcGo29R16YsrkRVbL62EFyXLnoOoMeEu6
Downloading:  https://canvas.csun.edu/files/5619026/download?download_frd=1&verifier=NY9O8RqjDtyf57ftXvkMDuGAF3BfmlYdPXp9iSzc
Downloading:  https://canvas.csun.edu/files/5618296/download?download_frd=1&verifier=zdUSLGz9bI7EEJT3eCAf9GPaobM4m9JHws412Mwi
Downloading:  https://canvas

Downloading:  https://canvas.csun.edu/files/5663985/download?download_frd=1&verifier=kGgkWHVnlghrqe0irKgvQa2wik9NBB9piiUqrCsi
Downloading:  https://canvas.csun.edu/files/5666597/download?download_frd=1&verifier=gdD4RxVDGCdxYg75cwCwObbn6IsIjBU2k8iKrqQu
Downloading:  https://canvas.csun.edu/files/5667657/download?download_frd=1&verifier=h1nmZyFVSqaC7ckreJAXAENkBfS9G51ukV3Ttcyg
Downloading:  https://canvas.csun.edu/files/5663794/download?download_frd=1&verifier=xrN6BSoCVaCaIVIJ525sOPcrnMcp0wpCSpN9K9RE
Downloading:  https://canvas.csun.edu/files/5659991/download?download_frd=1&verifier=HMOQIkGhsnSeaCrFBfmEM1FoJV2eck8O1SsOYmHc
Downloading:  https://canvas.csun.edu/files/5667811/download?download_frd=1&verifier=jySwkcMObLjNgFPILs9kuom9sEQ5BV5wcQanV1cn
Downloading:  https://canvas.csun.edu/files/5667777/download?download_frd=1&verifier=3D53UQuyQeDFoEJvZEqWJczKTuIp9Pj7iJmg4Pfq
Downloading:  https://canvas.csun.edu/files/5662255/download?download_frd=1&verifier=uS3oCTBbCWWhPE5rHQFOCFBuiuIy2zDxy

# Read student work and check that properly categorized

In [4]:
from ipywidgets import widgets
from IPython.display import display

def make_assignment_header(store):
    """Displays the header for each assignment"""
    entry = """
    <h1>{}</h1>
    <h2>Class id: {}</h2>"""
    return display(widgets.HTML(entry.format(store.assignment_name, store.course_id)))


def make_text_display(student_id, text):
    """Handles the formatting of the student's text"""
    entry = """
          <div id='%s'>
            <h3>%s</h3>
            <p>
                %s
            </p>
        </div>"""
    e = entry % (student_id, student_id, text)
    return widgets.HTML(e)

def make_submission_output(text, student_id, credit_list):
    """Creates the display of the submitted text with a toggle button to 
    update whether the student receives credit 
    """
    bval = 'Credit' if student_id in credit_list else 'No credit'
    credit_button = widgets.ToggleButtons(
        options=['Credit', 'No credit'],
        value=bval,
        description='',
        disabled=False
    )
    
    ta = make_text_display(student_id, text)

    def on_value_change(change):
        """The relevant callback
        NB the use of scope to define student_id"""
        v = change['new']
        print(v, student_id)
        try:
            if v == 'Credit':
                credit_list.append(student_id)
            elif v == 'No credit':
                credit_list.remove(student_id)
        except ValueError:
            pass

    credit_button.observe(on_value_change, names='value')
    display(widgets.VBox([ta, credit_button]))

def make_consolidated_text_fields(store):
    """Displays each entry with a toggle to adjust whether the 
    student receives credit"""
    for r in store.submissions:
        make_submission_output(r['body'], r['student_id'], store.credit)

In [ ]:
test_results = [{'student_id' : i, 'body': 'Body text for student {}'.format(i)} for i in range(0,4)]

# credit_list = [0, 2]

# make_consolidated_text_fields(test_results, credit_list)

In [5]:
for s in results:
    make_assignment_header(s)
    make_consolidated_text_fields(s)

HTML(value='\n    <h1>Journal (week 1)</h1>\n    <h2>Class id: 67531</h2>')

HTML(value='\n    <h1>Journal (week 2)</h1>\n    <h2>Class id: 67531</h2>')

# Upload the grades to canvas

In [10]:
from assets.UploadGradeTools import upload_students_receiving_credit
from ipywidgets import widgets
from IPython.display import display

def make_upload_button(store):
    def upload_callback(event):
        # change button style to indicate working
        b.button_style='warning'
        # upload grades for students receiving credit
        upload_students_receiving_credit(store)
        # change button style
        b.button_style='success'
        
    desc = 'Upload grades for {}'.format(store.assignment_name)
    layout= widgets.Layout(width='50%')
    b = widgets.Button(description=desc, button_style='danger', layout=layout)
    b.on_click(upload_callback)
    
    display(b)

In [7]:
for store in results:
    make_upload_button(store)

Button(button_style='danger', description='Upload grades for Journal (week 1)', layout=Layout(width='50%'), st…

Button(button_style='danger', description='Upload grades for Journal (week 2)', layout=Layout(width='50%'), st…

credit https://canvas.csun.edu/api/v1/courses/67531/assignments/360442/submissions/23662
credit https://canvas.csun.edu/api/v1/courses/67531/assignments/360442/submissions/25782
credit https://canvas.csun.edu/api/v1/courses/67531/assignments/360442/submissions/36128
credit https://canvas.csun.edu/api/v1/courses/67531/assignments/360442/submissions/36520
credit https://canvas.csun.edu/api/v1/courses/67531/assignments/360442/submissions/39746
credit https://canvas.csun.edu/api/v1/courses/67531/assignments/360442/submissions/39892
credit https://canvas.csun.edu/api/v1/courses/67531/assignments/360442/submissions/42021
credit https://canvas.csun.edu/api/v1/courses/67531/assignments/360442/submissions/43060
credit https://canvas.csun.edu/api/v1/courses/67531/assignments/360442/submissions/43081
credit https://canvas.csun.edu/api/v1/courses/67531/assignments/360442/submissions/44219
credit https://canvas.csun.edu/api/v1/courses/67531/assignments/360442/submissions/44270
credit https://canvas

credit https://canvas.csun.edu/api/v1/courses/67531/assignments/360449/submissions/44270
credit https://canvas.csun.edu/api/v1/courses/67531/assignments/360449/submissions/44360
credit https://canvas.csun.edu/api/v1/courses/67531/assignments/360449/submissions/45005
credit https://canvas.csun.edu/api/v1/courses/67531/assignments/360449/submissions/45309
credit https://canvas.csun.edu/api/v1/courses/67531/assignments/360449/submissions/45429
credit https://canvas.csun.edu/api/v1/courses/67531/assignments/360449/submissions/46335
credit https://canvas.csun.edu/api/v1/courses/67531/assignments/360449/submissions/46791
credit https://canvas.csun.edu/api/v1/courses/67531/assignments/360449/submissions/48809
credit https://canvas.csun.edu/api/v1/courses/67531/assignments/360449/submissions/53606
credit https://canvas.csun.edu/api/v1/courses/67531/assignments/360449/submissions/54653
credit https://canvas.csun.edu/api/v1/courses/67531/assignments/360449/submissions/55124
credit https://canvas